CS524: Introduction to Optimization Lecture 10
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## September 25, 2024
--------------

#  Shortest Path


In [1]:
import sys
import pandas as pd
import numpy as np

from gamspy import (
    Container,Set,Alias,Parameter,Variable,Equation,Model,Problem,Sense,Options,
    Domain,Number,Sum,Product,Smax,Smin,Ord,Card,SpecialValues,
    ModelStatus,SolveStatus,
)
from gamspy.exceptions import GamspyException
import gamspy.math as gpm
import math

options = Options(variable_listing_limit=100, equation_listing_limit=8,seed=101)
m = Container(options=options)

In [2]:
# DATA #
node = Set(m,'node',records=[f"node{i}" for i in range(1,9)])
i = Alias(m,'i',node)
j = Alias(m,'j',node)
k = Alias(m,'k',node)
coords = Set(m,'coords', records=['x','y'])

supply = Parameter(m,'supply',[i],records=[('node1',1),('node8',-1)])
if np.sum(supply.toDense()) != 0:
    raise GamspyException("supply must equal demand", supply)

posn = Parameter(m,'posn',[i,coords],records=pd.DataFrame(
    data = [
    ('node1',0,1),
    ('node2',0,0),
    ('node3',1,2),
    ('node4',1,0),
    ('node5',2,2),
    ('node6',2,1),
    ('node7',2,0),
    ('node8',3,1)], columns = ["i","x","y"]).set_index("i"),uels_on_axes=True)

# define a dynamic set that indicates the "legal" arcs
arc = Set(m,'arc',domain=[i,j],records=[
    ('node1','node2'),('node1','node3'),
    ('node2','node4'),
    ('node3','node6'),
    ('node4','node7'),
    ('node5','node8'),
    ('node6','node5'),('node6','node8'),
    ('node7','node6'),('node7','node8') ])
lakeCrossing = m.addSet('lakeCrossing',domain=[i,j],records=[
    ('node4','node7'),('node6','node8'),('node7','node6') ])

# check that lakeCrossing is a subset of arc
diff = Set(m,'diff',domain=[i,j])
diff[i,j] = lakeCrossing[i,j] - arc[i,j]
if diff.records is not None:
    raise GamspyException("Lake Crossings must be a subset of Arcs",diff)

# figure the distances
distance = Parameter(m,'distance',domain=[i,j])

distance[arc[i,j]] = gpm.sqrt(Sum(coords,gpm.sqr(posn[i,coords] - posn[j,coords])))
# double the distance if the route goes over a lake
distance[lakeCrossing] = 2*distance[lakeCrossing]

# VARIABLES #

x = Variable(m,"x","positive",domain=[i,j],description="flow")

# EQUATIONS #
# remove redundant constraint to get unique multipliers
balance = Equation(m,'balance',domain=[i])
balance[i].where[~i.last]= Sum(arc[i,k], x[i,k]) - Sum(arc[j,i], x[j,i]) == supply[i]
# balance[i].where[~i.sameAs('node8')]= Sum(k.where[arc[i,k]], x[i,k]) - Sum(j.where[arc[j,i]], x[j,i]) == supply[i]

short = Model(m,
    name="short",
    equations=[balance],
    problem=Problem.LP,
    sense=Sense.MIN,
    objective=Sum(arc, distance[arc]*x[arc]),
)

short.solve(solver='cplex',solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0, 'names': 'no'},output=None)

# POST PROCESSING #

print("Objective Function Value:  ", round(short.objective_value, 4), "\n")
print("x:\n", x.pivot())

Objective Function Value:   4.8284 

x:
        node2  node3  node4  node5  node6  node7  node8
node1    0.0    1.0    0.0    0.0    0.0    0.0    0.0
node2    0.0    0.0    0.0    0.0    0.0    0.0    0.0
node3    0.0    0.0    0.0    0.0    1.0    0.0    0.0
node4    0.0    0.0    0.0    0.0    0.0    0.0    0.0
node5    0.0    0.0    0.0    0.0    0.0    0.0    0.0
node6    0.0    0.0    0.0    0.0    0.0    0.0    1.0
node7    0.0    0.0    0.0    0.0    0.0    0.0    0.0


# Dual Model

In [3]:
pi = Variable(m,"pi","positive",domain=[i])

dualcons = Equation(m,'dualcons',domain=[i,j])
# dualcons[i,j].where[arc[i,j]]= pi[i] - pi[j] <= distance[i,j]
dualcons[arc[i,j]]= pi[i] - pi[j] <= distance[i,j]

dualflow = Model(m,
    name="dualflow",
    equations=[dualcons],
    problem=Problem.LP,
    sense=Sense.MAX,
    objective=Sum(i, supply[i]*pi[i]),
)

# fix dual multiplier corresponding to dropped constraint
# pi.fx['node8'] = 0

dualflow.solve(solver='cplex',solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0, 'names': 'no'},output=None)

# POST PROCESSING #
    
print("Dual objective Function Value:  ", round(dualflow.objective_value, 4), "\n")
print("pi:\n", pi.records[['i','level']])
print("balance.m:\n", balance.records[['i','marginal']])

Dual objective Function Value:   4.8284 

pi:
        i     level
0  node1  4.828427
1  node2  3.828427
2  node3  3.414214
3  node4  2.828427
4  node5  1.000000
5  node6  2.000000
6  node7  0.828427
7  node8  0.000000
balance.m:
        i  marginal
0  node1  4.828427
1  node2  3.828427
2  node3  3.414214
3  node4  2.828427
4  node5  1.000000
5  node6  2.000000
6  node7  0.828427


In [4]:
# %%timeit
# DATA #
options.variable_listing_limit=0
options.equation_listing_limit=0
options.suppress_compiler_listing=True
options.write_listing_file=False
options.generate_name_dict=False
options.seed=101

node = Set(m,'node',records=range(1,1001))

supply = Parameter(m,'supply',[i],records=[(1,1),(1000,-1)])
if np.sum(supply.toDense()) != 0:
    raise GamspyException("supply must equal demand", supply)

iter = m.addSet('iter',records=[f"iter{i}" for i in range(1,101)])
density = m.addParameter('density',records=0.002)
successful = 0
averageLength = 0
solveTime = 0

for cnt in iter.toList():
    arc[i,j] = Number(1).where[gpm.uniform(0,1) < density]
    arc[i,i] = False
    distance[i,j] = gpm.uniform(1,10).where[arc[i,j]]
    try:
        short.solve(load_symbols=[],options=options,solver='cplex',solver_options={'lpmethod': 3, 'netfind': 2, 'preind': 0, 'names': 'no'},output=None)
        solveTime += short.total_solve_time
        if short.status == ModelStatus.OptimalGlobal:
            successful += 1
            averageLength += short.objective_value
    except:
        pass # infeasible model already detected by GAMS
       
# POST PROCESSING #

averageLength = (averageLength / successful) if successful > 0 else math.inf
percentageSuccess = 100*successful / len(iter.records);
print(f"Arc Density {density.toValue()}")
print(f"Proportion of Successful Paths: {percentageSuccess}")
print(f"Average length of Successful Paths: {averageLength:.8f}")
print(f"Total solve time: {solveTime:10.2f}")

Arc Density 0.002
Proportion of Successful Paths: 59.0
Average length of Successful Paths: 49.00945212
Total solve time:       1.86
